In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
There is no [mac64] chromedriver for browser 91.0.4472 in cache
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_mac64.zip
Driver has been saved in cache [/Users/robert/.wdm/drivers/chromedriver/mac64/91.0.4472.101]


# NASA Mars News



In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [5]:
# Retrieve the latest news title
news_title=soup.find_all('div', class_='content_title')[0].text
# Retrieve the latest news paragraph
news_p=soup.find_all('div', class_='rollover_description_inner')[0].text
news_title
news_p

'A mission to investigate key questions about potential life on Mars. '

# PL Mars Space Images - Featured Image

In [6]:
jpl_url="spaceimages-mars.com"
jpl_image_url="https://spaceimages-mars.com/"
browser.visit(jpl_image_url)


In [7]:
# HTML object
html=browser.html
# Parse HTML
soup=bs(html,"html.parser")
# Retrieve image url
image_url=soup.find_all('article')


In [8]:
image_url=soup.find('article')['style']
image_url=image_url.split("'")[1]

TypeError: 'NoneType' object is not subscriptable

In [10]:
featured_image_url=jpl_url+image_url
featured_image_url

TypeError: can only concatenate str (not "ResultSet") to str

# Mars Facts

In [11]:
# Scrape Mars facts from https://space-facts.com/mars/
url='https://galaxyfacts-mars.com/'
db=pd.read_html(url)
db

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [12]:
mars_db=db[0]
mars_db=mars_db.rename(columns={0:"Profile",1:"Value"},errors="raise")
mars_db.set_index("Profile",inplace=True)
mars_db

,Value,2
Profile,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [13]:
mars_db.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n      <th>2</th>\n    </tr>\n    <tr>\n      <th>Profile</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</tab

# Mars Hemispheres

In [14]:
url ='https://marshemispheres.com/'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=49784): Max retries exceeded with url: /session/e69930845e57d357d2098fd24518f62f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc0544f8370>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [15]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#url to scrap
url = 'https://marshemispheres.com/'
browser.visit(url)

hemisphere_image_urls = []

for i in range(0, 4):
    
    links_found = browser.links.find_by_partial_text('Hemisphere Enhanced')
    links_found[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    
    title = soup.find('h2', class_= 'title').text
    
    img_url = soup.find('div', class_= 'downloads')
    img_url = img_url.find('a', target= '_blank')['href']
    img_url = 'https://marshemispheres.com/' + img_url
    
    hemisphere_image_urls.append({
        'Title': title,
        'Image URL': img_url
    })
    
    browser.links.find_by_partial_text('Back').click()

#End browser session
browser.quit()



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/robert/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
